In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pywt
from scipy.signal import butter, sosfiltfilt, find_peaks, detrend
from scipy.integrate import cumtrapz
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [4]:
def get_contact_points(vaccel):
    vacc = detrend(vaccel)
    
    sos = butter(4, 2 * 20 / 50, output='sos')
    fvacc = sosfiltfilt(sos, vacc)
    
    vpos = cumtrapz(fvacc, dx=1/50, initial=0)
    
    # diff 1
    coef1, freq1 = pywt.cwt(vpos, np.arange(1, 50), 'gaus1', sampling_period=1/50)
    
    # get most prominent frequency
    F = np.abs(np.fft.rfft(coef1[8]))
    f_step = np.fft.rfftfreq(coef1.shape[1])[np.argmax(F)] * 50
    
    f_ic_opt = 0.69 * f_step + 0.34
    f_fc_opt = 3.6 * f_step - 4.5
    
    idx1 = np.argmin(np.abs(freq1 - f_ic_opt))
    idx2 = np.argmin(np.abs(freq1 - f_fc_opt))
    
    vacc_id = coef1[idx1]
    ic, *_ = find_peaks(-vacc_id, height=0.5 * np.std(vacc_id))
    
    coef2, _ = pywt.cwt(coef1[idx2], idx2, 'gaus1')
    vacc_idd = coef2[0]
    
    fc, *_ = find_peaks(vacc_idd, height=0.5*np.std(vacc_idd))
    
    return ic, fc, vacc_id, vacc_idd, fvacc

In [5]:
data = np.loadtxt('sample_data/sample2.csv', delimiter=',')
ftime, facc = data[:, 0], data[:, 1:4]
ftime -= ftime[0]

In [6]:
f, ax = plt.subplots(figsize=(6, 3))
ax.plot(facc)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
i1, i2 = 21000, 28000
i1, i2 = 8350, 9850  # sample 2
ds = 4
ds = 2  # sample 2
time, acc = ftime[i1:i2:ds], facc[i1:i2:ds]  # downsample to 50Hz as well

In [9]:
ic, fc, va_id, va_idd, vaf = get_contact_points(acc[:, 1])

In [11]:
f, ax = plt.subplots(figsize=(8, 4))

ax.plot(time, vaf, label='Accel.')
ax.plot(time, va_id, label='ID')
ax.plot(time, va_idd, label='IDD')
ax.plot(time[ic], va_id[ic], 'ko', label='IC')
ax.plot(time[fc], va_idd[fc], 'k+', label='FC')

ax.legend()
f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# GeneActiv/GaitMat Data

In [12]:
import inertial_sensor_routines as isr

ModuleNotFoundError: No module named 'inertial_sensor_routines'